In [4]:
import os
import pandas as pd
import numpy as np
import natsort
pd.set_option('display.max_rows', None)
from decimal import *
getcontext().prec = 2

from tkinter  import * 
import tkinter as tk
import tkinter.messagebox as messagebox
from functools import partial

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import math
import time

In [5]:
STORE_MAP = {
    '-Fossil Tmall domestic watch flagship store' : 'Fossilwatch',
    '-Fossil Tmall domestic Leather flagship store' : 'Fossilleather',
    '-Fossil Tmall International Store ' : 'Fossil Tmall Intl.',
    '-Emporio Armani Tmall domestic watch flagship store' : 'EAwatch',
    '-Emporio Armani Tmall domestic jewelry flagship store' : 'EAJewelry',
    '-Emporio Armani Tmall international watch flagship store' : 'EA Tmall Intl.',
    '-Diesel Tmall domestic watch flagship store' : 'DZ',
    '-Diesel Tmall international watch flagship store' : 'DZ Tmall Intl.',
    '-Michael Kors Tmall domestic watch flagship store' : 'MK',
    '-Michael Kors Tmall international watch flagship store' : 'MK Tmall Intl.',
    '-AX Tmall domestic watch flagship store' : 'AX',
    '-AX Tmall international watch flagship store' : 'AX Tmall Intl.',
    '-Skagen Tmall domestic watch flagship store' : 'Skagen',
    '-Puma Tmall domestic watch flagship store':'Puma',
    '-Kate Spade Tmall domestic watch flagship store': 'Kate Spade',
    '-WSI Tmall domestic watch flagship store' : 'WSI',
    '-WSI Tmall international watch flagship store': 'WSI Tmall Intl.',
    'TOTAL WSI WECHAT STORE - DIRECT' : 'WSI Wechat Store'
}
STORE_DICT = dict(zip(STORE_MAP.values(), STORE_MAP.keys()))

In [6]:
DATE_MAP = {
    'P1' : [1,2,3,4],
    'P2' : [5,6,7,8],
    'P3' : [9,10,11,12,13],
    'P4' : [14,15,16,17],
    'P5' : [18,19,20,21],
    'P6' : [22,23,24,25,26],
    'P7' : [27,28,29,30],
    'P8' : [31,32,33,34],
    'P9' : [35,36,37,38,39],
    'P10': [40,41,42,43],
    'P11': [44,45,46,47],
    'P12': [48,49,50,51,52]
}

In [7]:
def week_to_p(wk):
    for p in DATE_MAP:
        if wk in DATE_MAP.get(p):
            return p

In [9]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2 import service_account
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
# The ID and range of a sample spreadsheet.
sheetid = '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY'
p = week_to_p(27)
letter = 'B'
RANGE = '2021 ' + p + '!' + letter + '5:' + letter + '5'
SERVICE_ACCOUNT_FILE = 'mycredentials.json'
credentials = None
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

service = build('sheets', 'v4', credentials=credentials)

# Call the Sheets API
sheet = service.spreadsheets()
# result = sheet.values().get(spreadsheetId=sheetid, range=RANGE).execute()

store_cell = '2021 ' + p + '!A' + str(8)+ ':A' + str(8)
result = sheet.values().get(spreadsheetId=sheetid, range=store_cell).execute()
values = result.get('values', [])
store = values[0][0]
store

'Fossil Total'

In [10]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2 import service_account


def update_sheet(cell,value,p):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    # The ID and range of a sample spreadsheet.
    sheetid = '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY'
    RANGE = '2021 '+p+'!A5:O117'

    SERVICE_ACCOUNT_FILE = 'mycredentials.json'
    credentials = None
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=credentials)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=sheetid, range=RANGE).execute()
    values = result.get('values', [])
    #     print(values)
    if math.isnan(value):
        value = 0
    values = [[value]]
    
    body = {'values': values}
    request = sheet.values().update(spreadsheetId=sheetid,
                                    range=cell,
                                    valueInputOption='USER_ENTERED',
                                    body=body)
    response = request.execute()

    # TODO: Change code below to process the `response` dict:
    print(response)

In [17]:
def update_wk(wk_num):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    # The ID and range of a sample spreadsheet.
    sheetid = '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY'
    p = week_to_p(wk_num)

    SERVICE_ACCOUNT_FILE = 'mycredentials.json'
    credentials = None
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=credentials)

    # Call the Sheets API
    sheet = service.spreadsheets()

    file_21 = pd.read_excel('../../test/Direct draft for Ruiyang.xlsx',
                            sheet_name='21',
                            nrows=20)
    file_20 = pd.read_excel('../../test/Direct draft for Ruiyang.xlsx',
                            sheet_name='20',
                            nrows=20)

    df21 = pd.DataFrame(file_21)
    df20 = pd.DataFrame(file_20)

    for i in range(8, 40):
        store_cell = '2021 '+p+'!A' + str(i) + ':A' + str(i)
        #         count_cell = '2021 P2!E' + str(i) + ':E' + str(i)
        result = sheet.values().get(spreadsheetId=sheetid,
                                    range=store_cell).execute()
        values = result.get('values', [])
        store = values[0][0]
        
        if store in STORE_MAP:
            offline_store = STORE_MAP[store]
            index = df21[df21['2021 Net Sales'] == offline_store].index[0]

            #             need to update online file from B to E or F
            five_wk_p = ['P3', 'P6', 'P9', 'P12']
            if p in five_wk_p:
                column_letter = ['B', 'C', 'D', 'E', 'F']
            else:
                column_letter = ['B', 'C', 'D', 'E']
            for letter in column_letter:
                wk_cell = '2021 '+p+'!'+letter+'5:'+letter+'5'
                temp_result = sheet.values().get(spreadsheetId=sheetid,
                                                 range=wk_cell).execute()
                temp_values = temp_result.get('values', [])
                wk = temp_values[0][0]
                # parameter wk is from online sheet.\
                if wk in df21:
                    count = df21[wk][index]
                    count_cell = '2021 ' + p + '!' + letter + str(
                        i) + ':' + letter + str(i)
                    update_sheet(count_cell, count, p)
                else:
                    continue
        time.sleep(5)

In [18]:
'''
Also need 2021 QTD, and 2020 MTD QTD
Directly import value of these from local file, value is counted by excel.
2021 MTD is not needed, because it can be counted directly from online sheet.
'''

def update_td(wk_num):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    # The ID and range of a sample spreadsheet.
    sheetid = '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY'
    p = week_to_p(wk_num)

    SERVICE_ACCOUNT_FILE = 'mycredentials.json'
    credentials = None
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('sheets', 'v4', credentials=credentials)

    # Call the Sheets API
    sheet = service.spreadsheets()

    file_21 = pd.read_excel('../../test/Direct draft for Ruiyang.xlsx',sheet_name = '21',nrows = 20)
    file_20 = pd.read_excel('../../test/Direct draft for Ruiyang.xlsx',sheet_name = '20',nrows = 20)

    df21 = pd.DataFrame(file_21)
    df20 = pd.DataFrame(file_20)

    five_wk_p = ['P3', 'P6', 'P9', 'P12']
    if p in five_wk_p:
        col_21_qtd = 'J'
        col_21_mtd = 'G'
        col_20_qtd = 'K'
        col_20_mtd = 'H'
        col_21_ytd = 'M'
        col_20_ytd = 'N'
    else:
        col_21_qtd = 'I'
        col_21_mtd = 'F'
        col_20_qtd = 'J'
        col_20_mtd = 'G'
        col_21_ytd = 'L'
        col_20_ytd = 'M'
    for i in range(8,40):
        store_cell = '2021 '+p+'!A' + str(i) + ':A' + str(i)
        result = sheet.values().get(spreadsheetId=sheetid, range=store_cell).execute()
        values = result.get('values', [])
        store = values[0][0]
        if store in STORE_MAP:
            offline_store = STORE_MAP[store]
            index_21 = df21[df21['2021 Net Sales'] == offline_store].index[0]
            index_20 = df20[df20['2020 Net Sales'] == offline_store].index[0]
            # first, update 2021 QTD
            count_qtd_21 = df21['QTD'][index_21]
            # 2021 QTD cell supose to always be in cell I? NONO
            # Need to find the column of MTD and QTD.
            # If P include 5 wk, then 21 QTD column is J, if not, I.
            
            qtd_21_cell = '2021 '+p+'!'+col_21_qtd+str(i)+':'+col_21_qtd+str(i)
            update_sheet(qtd_21_cell, count_qtd_21,p)
            
            # then, update 2020 MTD,QTD
            count_mtd_20 = df20['MTD'][index_20]
            count_qtd_20 = df20['QTD'][index_20]
            
            mtd_20_cell = '2021 '+p+'!'+col_20_mtd+str(i)+':'+col_20_mtd+str(i)
            qtd_20_cell = '2021 '+p+'!'+col_20_qtd+str(i)+':'+col_20_qtd+str(i)
            update_sheet(mtd_20_cell, count_mtd_20,p)
            update_sheet(qtd_20_cell, count_qtd_20,p)
            
            # update 2021 and 2020 YTD
            count_21_cell = df['YTD'][index_21]
            count_20_cell = df['YTD'][index_20]
            
            ytd_21_cell = '2021 '+p+'!'+col_21_ytd+str(i)+':'+col_21_ytd+str(i)
            ytd_20_cell = '2021 '+p+'!'+col_20_ytd+str(i)+':'+col_20_ytd+str(i)
            update_sheet(ytd_21_cell, count_ytd_21,p)
            update_sheet(ytd_20_cell, count_ytd_20,p)
        time.sleep(5)

In [22]:
# Test for wk
update_wk(27)

{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B9", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}


KeyboardInterrupt: 

In [20]:
# Test for td
# update_td(27)

In [23]:
# main function
def main(wk):
    update_wk(wk)
    update_td(wk)

if __name__ == "__main__":
    wk = int(input('Please enter the week number: '))
    main(wk)

Please enter the week number: 28
{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B9", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}
{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B10", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}
{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B11", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}
{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B13", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}
{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B14", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}
{'spreadsheetId': '1S6dgcPNrU18_8xB7LJVfSUpJekKTP08yhJwomQbHmlY', 'updatedRange': "'2021 P7'!B15", 'updatedRows': 1, 'updatedColumns': 1, 'updatedCells': 1}


KeyboardInterrupt: 